In [2]:
import json

In [ ]:
def filter_programs_by_electives(student_electives, programs):
    matched_programs = []

    for program in programs:
        elective_text = program.get("elective subjects", "")
        if not elective_text:
            continue

        qualifies = False

        # Normalize everything to lowercase for comparison
        elective_text_lower = elective_text.lower()
        student_electives_lower = [e.lower() for e in student_electives]

        if "three subjects from" in elective_text_lower:
            # Extract subjects from that list
            parts = elective_text_lower.split("three subjects from:")[-1]
            elective_options = [
                item.strip() for item in parts.replace('\n', ',').split(',')
                if item.strip()
            ]

            match_count = sum(
                any(opt in student_subject for opt in elective_options)
                for student_subject in student_electives_lower
            )

            qualifies = match_count >= 3

        else:
            # Handle specific electives or "A or B" type logic
            conditions = [cond.strip() for cond in elective_text_lower.split(", and ")]
            qualifies = all(
                any(opt.strip() in subject for subject in student_electives_lower)
                if " or " not in cond else
                any(
                    any(opt.strip() in subject for subject in student_electives_lower)
                    for opt in cond.split(" or ")
                )
                for cond in conditions
            )

        if qualifies:
            matched_programs.append({
                "program": program["program"],
                "college": program["college"],
                "required_electives": program["elective subjects"]
            })

    return matched_programs


# Example usage:
with open("programdata.json", "r", encoding="utf-8") as f:
    program_data = json.load(f)

student_electives = [
    "Physics", "Chemistry", "Elective Mathematics", "Biology"
]

matches = filter_programs_by_electives(student_electives, program_data)

# Display results
for program in matches:
    print(f"{program['program']} ({program['college']})")
    print("Requires:", program["required_electives"])
    print("-" * 40)
